In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Importation of the dataset
import pandas as pd
curve_trx_from_to_path = 'curve_trx_from_to.csv'  
veCRV_20holders_path = 'veCRV_20holders.csv'      
curve_trx_from_to = pd.read_csv(curve_trx_from_to_path)
veCRV_20holders = pd.read_csv(veCRV_20holders_path)

C:\Users\Badr\AppData\Local\Temp\ipykernel_20152\2883652382.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  curve_trx_from_to = pd.read_csv(curve_trx_from_to_path)


In [3]:
# Remove the row where the 'to' variable is missing in curve_trx_from_to dataset
curve_trx_from_to_cleaned = curve_trx_from_to.dropna(subset=['to'])

In [4]:
# Replace missing values in object columns with "Unknown"
object_columns = curve_trx_from_to_cleaned.select_dtypes(include=['object']).columns
curve_trx_from_to_cleaned[object_columns] = curve_trx_from_to_cleaned[object_columns].fillna('Unknown')

C:\Users\Badr\AppData\Local\Temp\ipykernel_20152\1406052502.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curve_trx_from_to_cleaned[object_columns] = curve_trx_from_to_cleaned[object_columns].fillna('Unknown')


In [5]:
curve_trx_from_to_cleaned['timeStamp'] = pd.to_datetime(curve_trx_from_to_cleaned['timeStamp'], unit='s')

# Then, set the timeStamp as the index of the DataFrame
curve_trx_from_to_cleaned.set_index('timeStamp', inplace=True)

C:\Users\Badr\AppData\Local\Temp\ipykernel_20152\534350133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curve_trx_from_to_cleaned['timeStamp'] = pd.to_datetime(curve_trx_from_to_cleaned['timeStamp'], unit='s')


In [6]:
# A. Wash Trading Analysis
# Look for multiple transactions where 'from' and 'to' addresses are the same
wash_trading_candidates = curve_trx_from_to_cleaned[curve_trx_from_to_cleaned['from'] == curve_trx_from_to_cleaned['to']]

In [8]:
# B. Pump-and-Dump Analysis
# Detect significant increases in trading volume and price
volume_spikes = curve_trx_from_to_cleaned['value'].rolling(window='1D').sum().diff().nlargest(3)
price_spikes = curve_trx_from_to_cleaned['gasPrice'].rolling(window='1D').mean().diff().nlargest(3)


In [9]:
# C. Account Behavior Analysis
# Look for accounts that are frequently involved in the largest transactions
top_traders = curve_trx_from_to_cleaned.groupby('from')['value'].sum().nlargest(10)


In [10]:
# D. Statistical Anomaly Detection
# Use z-scores or other statistical measures to find outliers in transaction frequency and size
transaction_z_scores = (curve_trx_from_to_cleaned['value'] - curve_trx_from_to_cleaned['value'].mean()) / curve_trx_from_to_cleaned['value'].std()
outliers = curve_trx_from_to_cleaned[transaction_z_scores.abs() > 3]

In [11]:
# Print the findings
print("Potential wash trading transactions:")
print(wash_trading_candidates)
print("\nVolume spikes that might indicate pump-and-dump:")
print(volume_spikes)
print("\nPrice spikes that might indicate market manipulation:")
print(price_spikes)
print("\nAccounts that could be top traders or manipulators:")
print(top_traders)
print("\nOutlier transactions that could indicate unusual activities:")
print(outliers)

Potential wash trading transactions:
Empty DataFrame
Columns: [Unnamed: 0, blockNumber, hash, nonce, blockHash, transactionIndex, from, to, value, gas, gasPrice, isError, txreceipt_status, input, contractAddress, cumulativeGasUsed, gasUsed, confirmations, methodId, functionName]
Index: []

Volume spikes that might indicate pump-and-dump:
timeStamp
2021-02-14 23:59:10    1.293500e+19
2021-09-11 01:43:55    1.020000e+19
2020-08-14 01:32:17    5.000000e+18
Name: value, dtype: float64

Price spikes that might indicate market manipulation:
timeStamp
2020-08-13 21:48:24    1.718182e+10
2022-01-22 11:31:33    1.467746e+10
2020-08-13 08:01:55    1.000000e+10
Name: gasPrice, dtype: float64

Accounts that could be top traders or manipulators:
from
0xb7c243a26a07e60fdf35ac2240e69d6446afd7a5    1.293500e+19
0xb5d85cbf7cb3ee0d56b3bb207d5fc4b82f43f511    1.020000e+19
0x823b92d6a4b2aed4b15675c7917c9f922ea8adad    1.000000e+19
0xdbe4033d88de63961720fc604c8aa8452eb34308    1.450000e+18
0x3c8b02b5b18e7c